In [ ]:
import pandas as pd
from datetime import datetime
from pathlib import Path
from tqdm import gui, tqdm
import os
import numpy as np

In [3]:
cwd = Path.cwd()
file_path = Path("C:\\PICKLE\\")
param_df = pd.read_csv("parameter.csv")
param_df["start_date"] = pd.to_datetime(param_df["start_date"], dayfirst=True)
param_df["end_date"] = pd.to_datetime(param_df["end_date"], dayfirst=True)

In [ ]:
SLIPAGES = {
    "nifty": 0.01,
    "banknifty": 0.0125,
    "finnifty": 0.01,
    "midcpnifty": 0.0125,
    "sensex": 0.0125,
    "bankex": 0.0125,
}
STEPS = {
    "nifty": 1000,
    "banknifty": 5000,
    "finnifty": 1000,
    "midcpnifty": 1000,
    "sensex": 5000,
    "bankex": 5000,
    "spxw": 500,
    "xsp": 50,
}
PREFIX = {
    "nifty": "Nifty",
    "banknifty": "BN",
    "finnifty": "FN",
    "midcpnifty": "MCN",
    "sensex": "SX",
    "bankex": "BX",
}


In [ ]:
def get_gap(options):
    try:
        strike = options.scrip.str[:-2].astype(float).unique()
        strike.sort()
        differences = np.diff(strike)
        min_gap = float(differences.min())
        min_gap = int(min_gap) if min_gap.is_integer() else min_gap
        return min_gap
    except Exception as e:
        print(e)

In [ ]:
def get_file_date(file: Path) -> datetime:
    """
    Extract date from filename prefix: YYYY-MM-DD_xxx.pkl
    """
    try:
        date_str = file.stem.split("_")[0]  # take '2019-03-07'
        return datetime.strptime(date_str, "%Y-%m-%d")
    except Exception:
        return None


In [ ]:
def load_files_in_date_range(opt_file_path: Path, fut_file_path: Path, start_date: datetime, end_date: datetime):
    options_dict = {}
    for file in opt_file_path.iterdir():
        if file.is_file():
            file_date = get_file_date(file)
            if file_date and start_date <= file_date <= end_date:
                options_dict[file_date.date()] = file

    futures_dict = {}
    for file in fut_file_path.iterdir():
        if file.is_file():
            file_date = get_file_date(file)
            if file_date and start_date <= file_date <= end_date:
                futures_dict[file_date.date()] = file
                
    common_dates = sorted(set(options_dict.keys()) & set(futures_dict.keys()))
    options_list = [options_dict[d] for d in common_dates]
    futures_list = [futures_dict[d] for d in common_dates]
    return options_list, futures_list

In [ ]:
def get_straddle_strike(
    opt: pd.DataFrame,
    fut: pd.DataFrame,
    start_dt: pd.Timedelta,
    end_dt: pd.Timedelta,
    gap: int = 50,
    sd: int = 0,
    SDroundoff: bool = False,
):
    """AI is creating summary for get_straddle_strike

    Args:
        opt (pd.DataFrame): [option data]
        fut (pd.DataFrame): [future data]
        start_dt (pd.Timedelta): [start/entry time]
        end_dt (pd.Timedelta): [end/exit time]
        gap (int, optional): [gap is different between two next scrip in option data]. Defaults to 50.
        sd (int, optional): [description]. Defaults to 0.
        SDroundoff (bool, optional): [description]. Defaults to False.
    """
    valid_times = fut.loc[start_dt:end_dt].index
    # print(valid_times[:2])
    for current_dt in valid_times:
        try:
            future_price = fut.loc[current_dt, "close"]
            # print(future_price, current_dt)
            round_future_price = round(future_price / gap) * gap
            ce_scrip, pe_scrip = f"{round_future_price}CE", f"{round_future_price}PE"
            # print(ce_scrip, pe_scrip)
            ce_price, pe_price = (
                opt[
                    (opt["date_time"] == current_dt) & (opt["scrip"] == ce_scrip)
                ].close.iloc[0],
                opt[
                    (opt["date_time"] == current_dt) & (opt["scrip"] == pe_scrip)
                ].close.iloc[0],
            )
            # print(ce_price, pe_price)
            syn_future = ce_price - pe_price + round_future_price
            round_syn_future = round(syn_future / gap) * gap
            ce_scrip_list = [
                f"{round_syn_future}CE",
                f"{round_syn_future+gap}CE",
                f"{round_syn_future-gap}CE",
            ]
            pe_scrip_list = [
                f"{round_syn_future}PE",
                f"{round_syn_future+gap}PE",
                f"{round_syn_future-gap}PE",
            ]
            # print(ce_scrip_list, pe_scrip_list)
            scrip_index, min_value = None, float("inf")
            for i in range(3):
                try:
                    ce_price = opt[
                        (opt.index == current_dt) & (opt["scrip"] == ce_scrip_list[i])
                    ].close.iloc[0]
                    pe_price = opt[
                        (opt.index == current_dt) & (opt["scrip"] == pe_scrip_list[i])
                    ].close.iloc[0]

                    diff = abs(ce_price - pe_price)
                    if min_value > diff:
                        min_value = diff
                        scrip_index = i
                except:
                    pass
            ce_scrip, pe_scrip = ce_scrip_list[scrip_index], pe_scrip_list[scrip_index]
            ce_price, pe_price = (
                opt[(opt.index == current_dt) & (opt["scrip"] == ce_scrip)].close.iloc[
                    0
                ],
                opt[(opt.index == current_dt) & (opt["scrip"] == pe_scrip)].close.iloc[
                    0
                ],
            )
            sd_range = 0
            if sd:
                sd_range = (ce_price + pe_price) * sd

                if SDroundoff:
                    sd_range = round(sd_range / gap) * gap
                else:
                    sd_range = max(gap, round(sd_range / gap) * gap)

            ce_scrip, pe_scrip = (
                f"{int(ce_scrip[:-2])+sd_range}CE",
                f"{int(pe_scrip[:-2])}PE",
            )
            ce_price, pe_price = (
                opt[(opt.index == current_dt) & (opt["scrip"] == ce_scrip)].close.iloc[
                    0
                ],
                opt[(opt.index == current_dt) & (opt["scrip"] == pe_scrip)].close.iloc[
                    0
                ],
            )
            return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt
        except (TypeError, ValueError, KeyError, IndexError):
            print("error occured in get straddle strike")
            continue

    return None, None, None, None, None, None

In [ ]:
def get_one_om(fut, future_price=None, STEP=1000):
    future_price = fut["close"].iloc[0] if future_price is None else future_price
    return (int(future_price / STEP) * STEP) / 100

In [ ]:
def get_strangle_strike(
    opt,
    fut,
    start_time,
    end_time,
    gap=50,
    om=None,
    target=None,
    check_inverted=False,
    tf=1,
    index=None
):

    valid_times = fut.loc[start_time:end_time].index
    for current_dt in valid_times:
        try:
            future_price = fut.loc[current_dt, "close"]
            # future price = 17510.4
            one_om = get_one_om(fut, future_price, STEP=STEPS[index])
            # one om = 170.0
            target = one_om * om if target is None else target
            # target = 68.0
            target_od = (
                opt[(opt.index == current_dt) & (opt["close"] >= target * tf)]
                .sort_values(by=["close"])
                .copy()
            )

            ce_scrip = target_od.loc[
                target_od["scrip"].str.endswith("CE"), "scrip"
            ].iloc[0]
            pe_scrip = target_od.loc[
                target_od["scrip"].str.endswith("PE"), "scrip"
            ].iloc[0]
            # ce_scrip = 17550CE
            # pe_scrip = 17400PE

            ce_scrip_list = [
                ce_scrip,
                f"{int(ce_scrip[:-2])-gap}CE",
                f"{int(ce_scrip[:-2])+gap}CE",
            ]
            pe_scrip_list = [
                pe_scrip,
                f"{int(pe_scrip[:-2])-gap}PE",
                f"{int(pe_scrip[:-2])+gap}PE",
            ]
            # ce_scrip_list = ['17550CE', '17500CE', '17600CE']
            # pe_scrip_list = ['17400PE', '17350PE', '17450PE']

            call_list_prices, put_list_prices = [], []
            for z in range(3):
                try:
                    call_list_prices.append(
                        opt[
                            (opt["date_time"] == current_dt)
                            & (opt["scrip"] == ce_scrip_list[z])
                        ]["close"].iloc[0]
                    )
                except:
                    call_list_prices.append(0)
                    print(
                        "call list price is empty and my date time column is not present in your data"
                    )
                try:
                    put_list_prices.append(
                        opt[
                            (opt["date_time"] == current_dt)
                            & (opt["scrip"] == pe_scrip_list[z])
                        ]["close"].iloc[0]
                    )
                except:
                    put_list_prices.append(0)
                    print(
                        "put list price is empty and my date time column is not present in your data"
                    )

            call, put, min_diff = call_list_prices[0], put_list_prices[0], float("inf")
            target_2, target_3 = target * 2 * tf, target * 3
            diff = abs(put - call)

            required_call, required_put = None, None
            if (put + call >= target_2) & (min_diff > diff) & (put + call <= target_3):
                min_diff = diff
                required_call, required_put = call, put

            for i in range(1, 3):
                if (
                    (min_diff > abs(put_list_prices[i] - call))
                    & (put_list_prices[i] + call >= target_2)
                    & (put_list_prices[i] + call <= target_3)
                ):
                    min_diff = abs(put_list_prices[i] - call)
                    required_call, required_put = call, put_list_prices[i]

                if (
                    (min_diff > abs(call_list_prices[i] - put))
                    & (call_list_prices[i] + put >= target_2)
                    & (call_list_prices[i] + put <= target_3)
                ):
                    min_diff = abs(call_list_prices[i] - put)
                    required_call, required_put = call_list_prices[i], put

            ce_scrip, pe_scrip = (
                ce_scrip_list[call_list_prices.index(required_call)],
                pe_scrip_list[put_list_prices.index(required_put)],
            )
            ce_price, pe_price = (
                opt[(opt["date_time"] == current_dt) & (opt["scrip"] == ce_scrip)][
                    "close"
                ].iloc[0],
                opt[(opt["date_time"] == current_dt) & (opt["scrip"] == pe_scrip)][
                    "close"
                ].iloc[0],
            )
            if int(ce_scrip[:-2]) < int(pe_scrip[:-2]) and check_inverted:
                get_straddle_strike(fut, current_dt, end_time, opt)
            else:
                return ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt

        except Exception as e:
            print("error occured in get strangle strike", e)
            continue

    return None, None, None, None, None, None

In [ ]:
fut = pd.read_pickle(
    "C:\\PICKLE\\BN Future\\2025-08-04_banknifty_future.pkl"
).set_index("date_time")
fut.index = pd.to_datetime(fut.index)
opt = pd.read_pickle("C:\\PICKLE\\BN Options\\2025-08-04_banknifty.pkl").set_index(
    "date_time"
)
opt.index = pd.to_datetime(opt.index)
entry_time = pd.to_datetime("2025-08-04 09:20:00")
exit_time = pd.to_datetime("2025-08-04 15:25:00")
sl = 8
intra_sl = 20
om = 0.1

In [ ]:
ce_scrip, pe_scrip, ce_price, pe_price = get_strangle_strike(
    entry_time, exit_time, opt, fut, 100, om=om, target=None, tf=1, index="banknifty"
)
ce_price, pe_price, ce_scrip, pe_scrip

In [ ]:
opt = opt.loc[entry_time:exit_time]
ce_data = opt[(opt["scrip"] == ce_scrip)]
pe_data = opt[(opt["scrip"] == pe_scrip)]

In [ ]:
ce_pe_price = ce_price + pe_price
sl_price = ce_pe_price * (1 - sl / 100)
intra_sl_price = ce_pe_price * (1 - intra_sl / 100)
sl_price, ce_pe_price, intra_sl_price

In [ ]:
def check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price):
    try:
        sl_time = ce_data[ce_data.close + pe_data.close <= sl_price].index[0]
    except:
        sl_time = None
    try:
        intra_sl_time = ce_data[
            np.minimum(ce_data.high + pe_data.low, ce_data.low + pe_data.high)
            >= intra_sl_price
        ].index[0]

    except:
        intra_sl_time = None

    return sl_time, intra_sl_time

In [ ]:
def calculate_sl_intra(entry_time, ce_data, pe_data, sl, intra_sl):
    ce_pe_price = ce_data.loc[entry_time, "close"] + pe_data.loc[entry_time, "close"]
    sl_price = ce_pe_price * (1 - sl / 100)
    intra_sl_price = ce_pe_price * (1 - intra_sl / 100)

    return sl_price, ce_pe_price, intra_sl_price

In [ ]:
sl_time, intra_sl_time = check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price)
sl_time, intra_sl_time

In [ ]:
total_pnl = 0
sl_time, intra_sl_time = check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price)

while True:
    if sl_time is None and intra_sl_time is None:
        exit_ce_pe = ce_data.loc[exit_time, "close"] + pe_data.loc[exit_time, "close"]
        pnl = ce_pe_price - exit_ce_pe
        total_pnl += pnl
        break
    elif sl_time is not None and intra_sl_time is not None:
        if sl_time < intra_sl_time:
            pnl = ce_pe_price - sl_price
            total_pnl += pnl
            entry_time = sl_time
            sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(entry_time, ce_data, pe_data, sl, intra_sl)
            sl_time, intra_sl_time = check_sl_hit(ce_data.loc[entry_time:].iloc[1:], pe_data.loc[entry_time:].iloc[1:], sl_price, intra_sl_price)

        else:
            print("INTRA SL HIT ", intra_sl)
            pnl = ce_pe_price - intra_sl_price
            print(pnl)
            total_pnl += pnl
            break
    elif sl_time is not None and intra_sl_time is None:
        pnl = ce_pe_price - sl_price
        total_pnl += pnl
        entry_time = sl_time
        sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(entry_time, ce_data, pe_data, sl, intra_sl)
        sl_time, intra_sl_time = check_sl_hit(ce_data.loc[entry_time:].iloc[1:], pe_data.loc[entry_time:].iloc[1:], sl_price, intra_sl_price)
    else:
        print("INTRA SL HIT ", intra_sl)
        pnl = ce_pe_price - intra_sl_price
        print(pnl)
        total_pnl += pnl
        break
        

In [ ]:
def sre(index,start_date,end_date,start_time,end_time,om,sl,intra_sl):

    file_path = Path("C:\\PICKLE\\")
    opt_file_path = file_path / f"{PREFIX[index]} Options"
    fut_file_path = file_path / f"{PREFIX[index]} Future"

    opt_file_list, fut_file_list = load_files_in_date_range(
        opt_file_path, fut_file_path, start_date, end_date
    )
    meta= {
        "P_Strategy": "SRE",
        "P_Index": index,
        "P_StartTime": start_time,
        "P_EndTime": end_time,
        "P_OrderSide": "BUY",
        "P_SL": sl,
        "P_IntraSL": intra_sl,
        "P_OM": om
    }
    
    combinations = []
    for opt_file, fut_file in zip(opt_file_list, fut_file_list):
        fut = pd.read_pickle(fut_file).set_index("date_time")
        opt = pd.read_pickle(opt_file).set_index("date_time")
        fut.index = pd.to_datetime(fut.index)
        opt.index = pd.to_datetime(opt.index)
        fut["date_time"] = fut.index
        opt["date_time"] = opt.index
        
        metadata = meta.copy()
        metadata["Date"] = opt.index[0].date()
        metadata["day"] = pd.to_datetime(opt.index[0].date()).day_name()
        metadata["dte"] = opt.iloc[0].dte + 1

        gap = get_gap(opt)
        # if om!=0:
        ce_scrip, pe_scrip, ce_price, pe_price, future_price, current_dt = get_strangle_strike(opt,fut,start_time,end_time,gap,om,index=index) if om != 0 else get_straddle_strike(opt,fut,start_date,end_date,gap,)

        metadata["EntryTime"] = current_dt
        metadata["Future"] = future_price
        metadata["CE.Strike"] = ce_scrip
        metadata["CE.Price"] = ce_price
        metadata['PE.Strike'] = pe_scrip
        metadata['PE.Price'] = pe_price

        opt = opt.loc[current_dt:end_time]
        ce_data = opt[(opt["scrip"] == ce_scrip)]
        pe_data = opt[(opt["scrip"] == pe_scrip)]

        sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(
            entry_time, ce_data, pe_data, sl, intra_sl
        )
        metadata['SL'] = sl
        metadata["IntraSL"] = intra_sl_price

        sl_time, intra_sl_time = check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price)
        sl_count = 0
        total_pnl = 0

        for i in range(8):
            if i != 0:
                metadata[f"{i}.Decay.Flag"] = False
                metadata[f"{i}.Decay.Time"] = ""
            metadata[f"{i}.SL.Flag"] = False
            metadata[f"{i}.SL.Time"] = ""
            metadata[f"{i}.PNL"] = 0    
        if sl == 0:
            exit_ce_pe = ce_data.loc[end_time, "close"] + pe_data.loc[end_time, "close"]
            pnl = exit_ce_pe - ce_pe_price - SLIPAGES[index]*ce_pe_price
            metadata["0.PNL"] = pnl
            total_pnl += pnl
            return pnl,metadata

        while True:
            if sl_time is None and intra_sl_time is None:
                exit_ce_pe = ce_data.loc[end_time, "close"] + pe_data.loc[end_time, "close"]
                pnl = exit_ce_pe - ce_pe_price - SLIPAGES[index] * ce_pe_price
                total_pnl += pnl
                break
            elif sl_time is not None and intra_sl_time is not None:
                if sl_time < intra_sl_time:
                    pnl = sl_price - ce_pe_price - SLIPAGES[index] * ce_pe_price
                    total_pnl += pnl
                    entry_time = sl_time
                    sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(
                        entry_time, ce_data, pe_data, sl, intra_sl
                    )
                    sl_time, intra_sl_time = check_sl_hit(
                        ce_data.loc[entry_time:].iloc[1:],
                        pe_data.loc[entry_time:].iloc[1:],
                        sl_price,
                        intra_sl_price,
                    )
                    metadata[f"{sl_count}.SL.Flag"] = True
                    metadata[f"{sl_count}.SL.Time"] = str(sl_time)
                    metadata[f"{sl_count}.PNL"] = pnl
                    sl_count += 1
                else:
                    print("INTRA SL HIT ", intra_sl)
                    pnl = intra_sl_price - ce_pe_price - SLIPAGES[index] * ce_pe_price
                    print(pnl)
                    total_pnl += pnl
                    break
            elif sl_time is not None and intra_sl_time is None:
                pnl = sl_price - ce_pe_price - SLIPAGES[index] * ce_pe_price
                total_pnl += pnl
                entry_time = sl_time
                if ce_data.empty():
                    break
                sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(
                    entry_time, ce_data, pe_data, sl, intra_sl
                )
                sl_time, intra_sl_time = check_sl_hit(
                    ce_data.loc[entry_time:].iloc[1:],
                    pe_data.loc[entry_time:].iloc[1:],
                    sl_price,
                    intra_sl_price,
                )
                metadata[f"{sl_count}.SL.Flag"] = True
                metadata[f"{sl_count}.SL.Time"] = str(sl_time)
                metadata[f"{sl_count}.PNL"] = pnl
                sl_count += 1
            else:
                print("INTRA SL HIT ", intra_sl)
                pnl = intra_sl_price - ce_pe_price - SLIPAGES[index] * ce_pe_price
                print(pnl)
                total_pnl += pnl
                break
        combinations.append(metadata) 
    return total_pnl, combinations

In [ ]:
processed_files = cwd / "output"
processed_files.mkdir(exist_ok=True)
for idx, rows in param_df.iterrows():
    # print(rows)
    index = rows['index'].lower()
    start_date = rows["start_date"]
    end_date = rows["end_date"]
    start_time = rows["start_time"]
    end_time = rows["end_time"]
    sl = rows["sl"]
    intra_sl = rows["intra_sl"]
    om = rows["om"]

    if sl > intra_sl:
        print("SL cannot be greater than Intra SL")
        continue

    total_pnl, combinat = sre(
        index, start_date, end_date, start_time, end_time, om, sl, intra_sl
    )
    pd.DataFrame(combinat).to_csv(f"{processed_files}\\sre {index} {sl} {intra_sl} {om}.csv", index=False)
    

0.1
